In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
X,y = make_classification(n_samples=1000, n_features=10,n_informative=2,n_redundant=8, weights=[0.9,0.1], flip_y=0,random_state=42)

np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [4]:
params={
    "solver": "lbfgs",
    "max_iter":1000,
    "multi_class": "auto",
    "random_state": 8888,
}

lr = LogisticRegression(**params)
lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)

report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [5]:
report_dict = classification_report(y_test,y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.956043956043956,
  'support': 270},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.915995115995116,
  'support': 300}}

In [6]:
import mlflow

In [51]:
mlflow.set_experiment("First Experiemnt")
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'fl_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression")

MlflowException: Cannot set a deleted experiment 'First Experiemnt' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

now a example

Exp 1 : train Logistic Rgresion classifier

In [7]:
log_reg = LogisticRegression(C=1,solver='liblinear')
log_reg.fit(X_train,y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test,y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



Exp 2 : train Random forest classifier

In [8]:
rf_clf = RandomForestClassifier(n_estimators=30,max_depth=3)
rf_clf.fit(X_train,y_train)
y_pred_rf_clf = rf_clf.predict(X_test)
print(classification_report(y_test,y_pred_rf_clf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



Exp 3 : train XGBoost classifier

In [9]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric='logloss')
xgb_clf.fit(X_train,y_train)
y_pred_xgb_clf = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb_clf))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



Exp 4 : handle class imbalance using SMOTE Tomek and then train XGboost

In [10]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res,y_train_res = smt.fit_resample(X_train,y_train)
np.unique(y_train_res,return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [11]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric='logloss')
xgb_clf.fit(X_train_res,y_train_res)
y_pred_xgb_clf = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb_clf))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



Track exp using MLFlow

In [12]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [13]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [14]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669724,
   'support': 270},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.9132610508757297,
   'support': 300}},
 {'0': {'precision': 0.9640287769784173,
   'recall': 0.9925925925925926,
   'f1-score': 0.9781021897810218,
   'support': 270},
  '1': {'precision': 0.9090909090909091,
   'recall': 0.6666666666666666,
   'f1-score': 0.7692307692307692,
   'support': 30},
  'accuracy': 0.96,
  'macro avg': {'precision': 0.9365598430346631,
   'recall': 0.8296296296296296,
   'f1-score': 0.8736664795058955,
   'support': 300},
  'weighted avg': {'precision': 0.95853499018966

In [17]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [18]:
# Initialize MLflow
mlflow.set_experiment("Anomaly_Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/06/05 08:29:24 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly_Detection' does not exist. Creating a new experiment.
2025/06/05 08:29:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/855046564834414223/runs/7be3d0faab0043d4a2d2c87e13804299
🧪 View experiment at: http://localhost:5000/#/experiments/855046564834414223


2025/06/05 08:29:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/855046564834414223/runs/2adf39ce8ece4edcb4cc22cf51e242a9
🧪 View experiment at: http://localhost:5000/#/experiments/855046564834414223


2025/06/05 08:29:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/855046564834414223/runs/1b6a4ba7ea3f4f07912e4491eee35457
🧪 View experiment at: http://localhost:5000/#/experiments/855046564834414223


2025/06/05 08:30:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/855046564834414223/runs/23c0cfc8b3704504b6fe2c34270a8e8a
🧪 View experiment at: http://localhost:5000/#/experiments/855046564834414223


## Register the model

In [19]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

result = mlflow.register_model(model_uri,model_name)

Please type RunID23c0cfc8b3704504b6fe2c34270a8e8a


Successfully registered model 'XGB-Smote'.
2025/06/05 08:31:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


## load the model

In [20]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [21]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}@Challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [22]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1749092750633, current_stage='None', description='', last_updated_timestamp=1749092750633, name='anomaly-detection-prod', run_id='23c0cfc8b3704504b6fe2c34270a8e8a', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [23]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@Champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])